In [6]:
SOURCE_IMG = "/kaggle/working/"
TARGET_IMG = "kaggle/working/"
RETRIEVE_DIR = "kaggle/working/"

# **split_image.py**

In [ ]:
import numpy as np

def split_image(img, patch_size=800):
    """
    Split image into patches of size (patch_size, patch_size).
    Returns a list of tuples: (patch, (x_offset, y_offset)).
    """
    H, W, C = img.shape
    patches = []

    n_h = H // patch_size
    n_w = W // patch_size
    rem_h = H % patch_size
    rem_w = W % patch_size

    for i in range(n_h):
        for j in range(n_w):
            y0 = i * patch_size
            x0 = j * patch_size
            patch = img[y0:y0+patch_size, x0:x0+patch_size, :]
            patches.append((patch, (x0, y0)))

    # thừa ngang
    if rem_w > 0:
        for i in range(n_h):
            y0 = i * patch_size
            x0 = W - patch_size
            patch = img[y0:y0+patch_size, x0:W, :]
            patches.append((patch, (x0, y0)))

    # thừa dọc
    if rem_h > 0:
        for j in range(n_w):
            y0 = H - patch_size
            x0 = j * patch_size
            patch = img[y0:H, x0:x0+patch_size, :]
            patches.append((patch, (x0, y0)))

    # thừa cả hai
    if rem_h > 0 and rem_w > 0:
        y0 = H - patch_size
        x0 = W - patch_size
        patch = img[y0:H, x0:W, :]
        patches.append((patch, (x0, y0)))

    return patches



if __name__ == "__main__":
    img = np.random.randint(0, 1, (110, 110, 3), dtype=np.uint8)
    patches = split_image(img, 25)
    print("Số patch:", len(patches))


Số patch: 25


# **calibration.py**

In [ ]:
def radiometric_correction(img=None, metadata=None):
    """
    Radiometric Correction for a image

    Args:
        img: Input image (numpy array).
        metadata: Source image metadata

    Returns:
        img: Radiometric corrected image (numpy array)
    """
    return img
    
def geometric_correction(img=None, metadata=None):
    """
    Geometric Correction for a image

    Args:
        img: Input image (numpy array).
        metadata: Source image metadata

    Returns:
        img: Geometric corrected image (numpy array)
    """
    return img
    
def calibration(img=None, metadata=None):
    """
    Calibration for a image

    Args:
        img: Input image (numpy array).
        metadata: Source image metadata
    Returns:
        img: Calibrated image
    """
    img = geometric_correction(img, metadata)
    img = radiometric_correction(img, metadata)
    return img

# **detection.py**

In [6]:
def ship_detection(model=None, img=None):
    """
    Detect ships in an image using a YOLO model.

    Args:
        model: Pre-loaded YOLO model
        img: Input image (numpy array)

    Returns:
        detection_infors (list): A list containing bounding box coordinates and confidence scores.
            Each element is structured as:
                [xyxy, conf]
            where:
                - xyxy: numpy array (n, 4), coordinates of bounding boxes (x1, y1, x2, y2).
                - conf: numpy array (n,), confidence scores for each box.
    """
    results = model(img)
    detection_infors = []
    for r in results:
        boxes = r.boxes
        xyxy = boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
        conf = boxes.conf.cpu().numpy()
        a_box = [xyxy, conf]
        detection_infors.append(a_box)
    return detection_infors
    

# **alert.py**

In [1]:
import os
import cv2
import gc

def scan_images(img_source_dir):
    """
    Scan a directory for image files and return their paths.

    Args:
        img_source_dir : str
            Path to the directory containing images. Defaults to SOURCE_IMG.

    Returns:
        list of str
            A list of full file paths to all images found in the directory.
        """
    large_image_path = []
    for img_name in os.listdir(img_source_dir):
        img_path = os.path.join(img_source_dir, img_name)
        large_image_path.append(img_name)
    return large_image_path

def read_a_large_image(a_large_image_path):
    """
    Read a large image from the given file path.

    Args:
        a_large_image_path : str
            Path to the large image file.

    Returns:
        numpy.ndarray
            The loaded image as a NumPy array (H, W, C)
    """
    ##Something will be written soon.
    print("Read successfully:", a_large_image_path)
    return a_large_image

def bb_infors_of_large_image(a_large_image_path, model=None):   

    """
    Perform ship detection on a large images within a directory.
    Args:
        source_dir : str
            Path to the directory containing a large image
        model : object, optional
            Pre-loaded YOLO model used for ship detection.

    Returns:
        list
            A list of bounding box information this image
            Each element corresponds to one detected set of boxes and has the form:
            
            [xyxy, conf]
            
            where:
                - xyxy : numpy.ndarray of shape (n, 4)
                    Absolute pixel coordinates of bounding boxes in the large image
                    [(x1, y1, x2, y2), ...].
                - conf : numpy.ndarray of shape (n,)
                    Confidence scores for each bounding box.
    """
    bb_infors = []
    large_img = read_a_large_image(a_large_image_path)
    splited_large_img_arr = split_image(large_img)
    print("Splited successfully:", a_large_image_path)
    for a_part_img, (x_offset, y_offset) in splited_large_img_arr:
        # detect on a patch
        bb_infor = ship_detection(model, a_part_img)  
        # bb_infor: [ (xyxy, conf) ]
        # convert across this large image
        new_bb_infor = []
        for xyxy, conf in bb_infor:
            xyxy[:, [0,2]] += x_offset  # cộng offset theo trục x
            xyxy[:, [1,3]] += y_offset  # cộng offset theo trục y
            new_bb_infor.append([xyxy, conf])
        bb_infors.extend(new_bb_infor)
    del large_img 
    del splited_large_img_arr
    gc.collect()
    return bb_infors

def format_alert(source_dir, targer_dir, model = None):
    """
    Run ship detection on each large image and save results as telemetry text files.

    Args:
        source_dir : str, optional
            Path to the directory containing large images. Defaults to SOURCE_IMG.
        target_dir : str, optional
            Path to the directory where telemetry files will be saved. Defaults to TARGET_IMG.
        model : object, optional
            Pre-loaded YOLO model used for ship detection.

    Output:
        For each large image, a text file named [count].txt is written to target_dir,
        containing:
            - Image path
            - Number of detected ships
            - One line per bounding box: [x1, y1, x2, y2, conf]
    """
    count = 0
    for a_large_img_path in scan_images(source_dir):
        bb_infors = bb_infors_of_large_image(a_large_img_path, model)
        out_path = os.path.join(target_dir, f"{count}.txt")
        with open(out_path, "w") as f:
            f.write(f"{a_large_img_path}\n")
            f.write(f"{len(bb_infors)}\n")  # số lượng ship detection
            for xyxy, conf in bb_infors:
                for i in range(len(conf)):
                    x1, y1, x2, y2 = xyxy[i]
                    c = conf[i]
                    f.write(f"[{x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f}, {c:.4f}]\n")
        count += 1
        gc.collect()
    return count

# **utils.py**

In [ ]:
def main(source_dir = None, targer_dir = None, retrive_dir =None, model = None):
    while True:
        print("="*40)
        print("                MAIN MENU")
        print("="*40)
        print("1. Count number of images collected")
        print("2. List all collected images with detected ship count")
        print("3. Retrieve alls")
        print("0. Exit")
        
        print("\nYour choice:")
        print("="*40)
        try:
            op = int(input("Enter option: "))
        except ValueError:
            print("Invalid input! Please enter a number.")
            continue
        if op == 0:
            print("Exiting DONE")
            break
        elif op == 1:
            print("Number of image collected: ", len(os.listdir(targer_dir)))
        elif op == 2:
            print("Listing images")
            print_image_info(targer_dir)
            user_input = str(input("Do you want retrieve some images, e.g: Enter 1 5 6 8 to collect images 1, 5, 6, and 8 or 0 to back Menu :"))
            if user_input == "0":
                continue
            retrieve_images(user_input, targer_dir, retrive_dir)
        elif op == 3:
            retrieve_images("all", targer_dir, retrive_dir)
            
        else:
            print("⚠️  Invalid option, please try again.")


In [ ]:
import os
import shutil

def print_image_info(target_dir):
    txt_files = sorted(
        [f for f in os.listdir(target_dir) if f.endswith(".txt")],
        key=lambda x: int(os.path.splitext(x)[0])  # sắp theo số 1,2,3,...
    )

    for idx, file_name in enumerate(txt_files, start=1):
        file_path = os.path.join(target_dir, file_name)
        with open(file_path, "r") as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        if len(lines) < 2:
            continue  # bỏ qua file lỗi

        image_path = lines[0]
        num_ships = lines[1]

        print(f"{idx}. Image information: {image_path} | Number of detected ships: {num_ships}")

def retrieve_images(user_input, target_dir, retrieve_dir):
    """
    Retrieve selected images and their telemetry info based on user input.
    
    Args:
        user_input (str): A string containing indexes separated by spaces (e.g., "1 5 6 8")
                          or "all" to retrieve all available files.
        target_dir (str): Directory containing telemetry files like 1.txt, 5.txt, etc.
        retrieve_dir (str): Directory to save retrieved images and telemetry info.
    """
    # Ensure retrieve_dir exists
    os.makedirs(retrieve_dir, exist_ok=True)

    # Determine which indexes to process
    if user_input.strip().lower() == "all":
        indexes = [os.path.splitext(f)[0] 
                   for f in os.listdir(target_dir) if f.endswith(".txt")]
    else:
        indexes = user_input.strip().split()

    for idx in indexes:
        txt_file = os.path.join(target_dir, f"{idx}.txt")
        
        if not os.path.exists(txt_file):
            print(f"Warning: {txt_file} not found. Skipping.")
            continue

        # Read telemetry file
        with open(txt_file, "r") as f:
            lines = f.readlines()
        
        if not lines:
            print(f"Warning: {txt_file} is empty. Skipping.")
            continue

        # First line = image path
        image_path = lines[0].strip()
        
        # Extract image name (after last '/')
        image_name = image_path.split("/")[-1]

        # Copy image to retrieve_dir
        if os.path.exists(image_path):
            shutil.copy(image_path, os.path.join(retrieve_dir, image_name))
        else:
            print(f"Warning: Image {image_path} not found. Skipping image copy.")

        # Save telemetry info into [image_name].txt
        telemetry_out_path = os.path.join(retrieve_dir, f"{image_name}.txt")
        with open(telemetry_out_path, "w") as f_out:
            f_out.writelines(lines)

        print(f"Retrieved: {image_name} + telemetry")





In [5]:
while True:
        print("="*40)
        print("                MAIN MENU")
        print("="*40)
        print("1. Count number of images collected")
        print("2. List all collected images with detected ship count")
        print("3. Retrieve alls")
        print("0. Exit")
        
        print("\nYour choice:")
        print("="*40)
        try:
            op = int(input("Enter option: "))
        except ValueError:
            print("Invalid input! Please enter a number.")
            continue
        if op == 0:
            print("Exiting DONE")
            break
        elif op == 1:
            print("ok option 1")
        elif op == 2:
            print("Listing images")
            #print_image_info(targer_dir)
            user_input = str(input("Do you want retrieve some images, e.g: Enter 1 5 6 8 to collect images 1, 5, 6, and 8 or 0 to back Menu :"))
            if user_input == "0":
                continue
            #retrieve_images(user_input, targer_dir, retrive_dir)
        elif op == 3:
            print("ok option 3")
            #retrieve_images("all", targer_dir, retrive_dir)
            
        else:
            print("⚠️  Invalid option, please try again.")

                MAIN MENU
1. Count number of images collected
2. List all collected images with detected ship count
3. Retrieve alls
0. Exit

Your choice:


Enter option:  1


ok option 1
                MAIN MENU
1. Count number of images collected
2. List all collected images with detected ship count
3. Retrieve alls
0. Exit

Your choice:


Enter option:  2


Listing images


...

Listing images


Do you want retrieve some images, e.g: Enter 1 5 6 8 to collect images 1, 5, 6, and 8 or 0 to back Menu : 1 2 5 6 8


                MAIN MENU
1. Count number of images collected
2. List all collected images with detected ship count
3. Retrieve alls
0. Exit

Your choice:


Enter option:  0


Exiting DONE
